In [ ]:
%load_ext autoreload
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn import metrics
from datetime import datetime
from src.average_precision import calculate_score
import numpy as np
import pickle
from keras.preprocessing import sequence
%autoreload 2
from src.average_precision import calculate_delta

In [ ]:
X_train = pickle.load( open( "data/X_train.pickle", "rb" ) )
y_train = np.array(pickle.load( open( "data/y_train.pickle", "rb" ) ))

In [ ]:
X_train =sequence.pad_sequences(X_train, maxlen=5, dtype='float')

In [ ]:
y_train =calculate_delta(X_train, y_train)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                    X_train, y_train, test_size=0.20, random_state=42)

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [ ]:
model.fit(X_train, y_train, nb_epoch=10, batch_size=512)

In [ ]:
preds = np.round(model.predict_proba(X_test), 4)

In [ ]:
score = calculate_score(y_test, preds)
print("The LB score is: {0}".format(score))
print(metrics.classification_report(y_test, np.round(preds)))

In [ ]:
fp = "models/lstm_{0}_SCORE_{1}".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), np.round(score,5)) 
model.save(fp)

In [ ]:
model.summary()